In [1]:
import pymongo
import os, sys
root_dir = "/home/cs/Documents/pymarl/src"
sys.path = ['/home/cs/Documents/pymarl/src'] + sys.path

In [2]:
from config import mongodb

In [3]:
from config.mongodb import REGISTRY as mongo_REGISTRY
def get_mongo_db_client(conf_name, maxSevSelDelay=5000):    
    mongo_conf = mongo_REGISTRY[conf_name](None, None)
    db_url = mongo_conf["db_url"]
    db_name = mongo_conf["db_name"]
    client = pymongo.MongoClient(db_url, ssl=True, serverSelectionTimeoutMS=maxSevSelDelay)
    return client, client[db_name]

In [4]:
class MongoCentral():
    
    def __init__(self, *args, **kwargs):
        self.conf_names = kwargs["conf_names"]
        self.db = {}
        self._connect(self.conf_names)
        
    def _connect(self, conf_names):
        self.clients = {}
        for _name in conf_names:
            self.clients[_name], self.db[_name] = get_mongo_db_client(_name)
            
    def get_exp_names(self):
        #print("Loading keys...")
        names = []
        for key, db in self.db.items():
            query = db["runs"].distinct("config.name") # .find({"config":None})
            names.extend(query)
            print("Done Loading...")
        return names
    
    def get_tag_names(self, tag, bundle=True):
        import re
        names = []
        for key, db in self.db.items():            
            query = db.runs.find({"config.name":{'$regex':r'^BASE2(.*)'}}, {"config.name":1}) # .find({"config":None})
            names.extend([_q["config"]["name"] for _q in query])
            print("Done Loading...")
            
        if bundle: # bundle by experiment name
            bundle_dic = {}
            for name in names:
                tag, exp_name_time_stamp, repeat = name.split("__")
                exp_name = "_".join(exp_name_time_stamp.split("_")[:-1])
                if exp_name not in bundle_dic:
                    bundle_dic[exp_name] = []
                bundle_dic[exp_name].append(name) 
            return bundle_dic
        return names


In [5]:
mongo_central = MongoCentral(conf_names=["gandalf_pymarl"])

In [6]:
a = mongo_central.get_exp_names()
print(a)
print("--------------------------")
import pprint
b = mongo_central.get_tag_names("BASE2")
pprint.pprint(b)

Done Loading...
['config', 'coma_baseline_3m__repeat5', 'coma_baseline_3m__repeat4', 'coma_baseline_5m__repeat1', 'coma_baseline_3m__repeat1', 'coma_baseline_3m__repeat3', 'coma_baseline_3m__repeat2', 'coma_baseline_2d_3z__repeat2', 'coma_baseline_2d_3z__repeat1', 'coma_baseline_2d_3z__repeat3', 'coma_baseline_2d_3z__repeat5', 'coma_baseline_2d_3z__repeat4', 'coma_baseline_5m__repeat2', 'coma_baseline_5m__repeat3', 'coma_baseline_5m__repeat4', 'coma_baseline_5m__repeat5', 'iql_pp3x3_3agentcap', 'BASE2__coma_baseline_2d_3z_01/05/18-13:03:01__repeat1', 'BASE2__coma_baseline_3m_01/05/18-13:03:01__repeat1', 'BASE2__coma_baseline_5m_01/05/18-13:03:01__repeat1', 'BASE2__coma_baseline_2d_3z_01/05/18-13:03:01__repeat2', 'BASE2__coma_baseline_3m_01/05/18-13:03:01__repeat2', 'BASE2__coma_baseline_5m_01/05/18-13:03:01__repeat2', 'BASE2__coma_baseline_3m_01/05/18-13:03:01__repeat3', 'BASE2__coma_baseline_2d_3z_01/05/18-13:03:01__repeat3', 'BASE2__coma_baseline_5m_01/05/18-13:03:01__repeat3', 'BASE

In [7]:
def plot_bundle_basics(bundle, mongo_central, mode = ""): # mode: individual, all
    if mode in ["individual"]:
        # make separate plot for each key, showing all subkeys
        for _k, _list in bundle.items():
            for item in _list:
    elif mode in ["all"]: 
        # plot all keys on same plot, averaging over subkeys

        pass

In [ ]:
def plot_bundle_sc2(bundle, mongo_central, mode = ""): # mode: individual, all
    if mode in ["individual"]:
        # make separate plot for each key, showing all subkeys
        for _k, _list in bundle.items():
            for item in _list:
    elif mode in ["all"]: 
        # plot all keys on same plot, averaging over subkeys

        pass